In [52]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv('spam_ham_dataset.csv')


In [53]:
from nltk.tokenize import RegexpTokenizer

def clean_str(string, reg = RegexpTokenizer(r'[a-z]+')):
    string = string.lower()
    tokens = reg.tokenize(string)
    return " ".join(tokens)

df['text_clean'] = df['text'].apply(lambda string: clean_str(string))
del df['text']
df = df[:2000]

In [60]:
str1 = '''        Subject: vocable % rnd - word asceticism
vcsc - brand new stock for your attention
vocalscape inc - the stock symbol is : vcsc
vcsc will be our top stock pick for the month of april - stock expected to
bounce to 12 cents level
the stock hit its all time low and will bounce back
stock is going to explode in next 5 days - watch it soar
watch the stock go crazy this and next week .
breaking news - vocalscape inc . announces agreement to resell mix network
services
current price : $ 0 . 025
we expect projected speculative price in next 5 days : $ 0 . 12
we expect projected speculative price in next 15 days : $ 0 . 15
vocalscape networks inc . is building a company that ' s revolutionizing the
telecommunications industry with the most affordable phone systems , hardware ,
online software , and rates in canada and the us . vocalscape , a company with
global reach , is receiving international attention for the development of voice
over ip ( voip ) application solutions , including the award - winning eyefontm , a
softphone for real - time pc - to - phone . we are an advanced implementer of pbx
systems for companies , call centers , itsps and service providers .
vocalscape has created software and interactive solutions revolving around
global communications and data voice convergence . companies use vocalscape for
voice over internet protocol applications like ip pbxs , softswitches , pc 2 phone
and web 2 phone , providing real - time human interaction and information delivery
over the internet . through vocalscape ' s solutions , businesses can offer a
quality voice service to anywhere in the world at rates that are significantly
lower than current long distance charges . we develop software to run voip
networks , and sell , install and service our own branded voip gateways and
gatekeeper control software . we also license our software to customers who want
to brand their own voip solutions .
vocalscape is committed to making great technology ; challenging the status quo ,
and building a 21 st century company that changes the way businesses communicate
and interact through the internet .
current price : $ 0 . 025
we expect projected speculative price in next 5 days : $ 0 . 12
we expect projected speculative price in next 15 days : $ 0 . 15
breaking news - vocalscape inc . announces agreement to resell mix network
services
- - - - - - - - - -
katonah , n . y . , / prnewswire - firstcall via comtex / - - vocalscape , inc . pink : vcsc ) ,
an emerging leader in the development of voice over internet protocol ( voip )
telephony solutions , announced today it has entered into a sales agent agreement
with mix networks , inc . a voip enhanced telephony service provider .
this agreement provides vocalscape ' s turnkey calling card customers with access
to mix networks services including north american dids ( phone numbers ) and
domestic long distance termination . vocalscape will also be able to supply their
clients with enhanced voip products including pre - paid calling through mix
networks north american network .
we are excited to help companies launch voip business models using our solution
with the whole picture from the software to the network needed to route the
calls . mix networks gives us the ability to offer wholesale monthly flat rate
plans and dids to our clients that allow for us to build business models like
popular voip companies such as vonage and packet 8 , says ryan gibson , vp
vocalscape networks
some legal words before you continue :
information within this email contains forward looking statements within the
meaning of section 27 a of the securities act of 1933 and section 21 b of the
securities exchange act of 1934 . any statements that express or involve
discussions with respect to predictions , goals , expectations , beliefs , plans ,
projections , objectives , assumptions or future events or performance are not
statements of historical fact and may be forward looking statements . forward
looking statements are based on expectations , estimates and projections at the
time the statements are made that involve a number of risks and uncertainties
which could cause actual results or events to differ materially from those
presently anticipated . forward looking statements in this action may be
identified through the use of words such as : projects , foresee , expects ,
estimates , believes , understands will , part of : anticipate
'''
len(str.encode(clean_str(str1)))


4095

In [48]:
[1,2,3,4,5][2:9]

[3, 4, 5]

In [29]:
# Convert a collection of text documents to a matrix of token counts
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(df.text_clean)

# Get the categories
y = df.label

# Split arrays or matrices into random train and test subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [30]:
# Find the best hyperparameter with GridSearchCV
# Exhaustive search over specified parameter values for an estimator.
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
parameters = {"alpha": [0.2,1,2,5,10], "fit_prior": [True, False]}

grid = GridSearchCV(MultinomialNB(), param_grid=parameters)
grid.fit(X_train,y_train)

# Create a DataFrame with the best Hyperparameters
pd.DataFrame(grid.cv_results_)[['params','mean_test_score']]\
                               .sort_values(by="mean_test_score", ascending=False)

# Create the model with the best hyperparameters
from sklearn.naive_bayes import MultinomialNB
alpha, fit_prior = grid.best_params_['alpha'], grid.best_params_['fit_prior']
model = MultinomialNB(alpha = alpha)

model.fit(X_train,y_train)
y_pred = model.predict(X_test)

from sklearn.metrics import classification_report, accuracy_score
print('Accuracy: ', round(accuracy_score(y_test,y_pred),3)*100)

Accuracy:  96.0


In [31]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         ham       0.98      0.97      0.97       306
        spam       0.90      0.94      0.92        94

    accuracy                           0.96       400
   macro avg       0.94      0.95      0.95       400
weighted avg       0.96      0.96      0.96       400

